In [16]:
import os
import random
import folders
from pydub import AudioSegment

In [2]:
def match_target_amplitude(sound, target_dBFS, offset_dB=5):
    change_in_dBFS = target_dBFS + offset_dB - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

def get_random_noise_file(noise_folder):
    noise_files = [f for f in os.listdir(noise_folder) if f.endswith('.wav')]
    return os.path.join(noise_folder, random.choice(noise_files))

In [3]:
def remove_silence(sound, silence_thresh=-50.0, min_silence_len=100):
    chunks = []
    start_time = None
    
    for i in range(0, len(sound), min_silence_len):
        segment = sound[i:i + min_silence_len]
        if segment.dBFS > silence_thresh:
            if start_time is None:
                start_time = i
        else:
            if start_time is not None:
                chunks.append(sound[start_time:i])
                start_time = None

    if start_time is not None:
        chunks.append(sound[start_time:])

    return sum(chunks) if chunks else AudioSegment.silent(duration=0)

In [18]:
train_folder = folders.train_folder
noise_folder = folders.noise_folder
save_folder = folders.save_folder

In [ ]:
train_audio = [file[2] for file in os.walk(train_folder)][0]
count = 0

for file_path in train_audio:
    count += 1
    clean_file = AudioSegment.from_wav(train_folder + file_path)
    
    clean_file = remove_silence(clean_file)
    clean_file.apply_gain(-7)
    
    noise_file_path = get_random_noise_file(noise_folder)
    noise_sound = AudioSegment.from_wav(noise_file_path)
    noise_sound = remove_silence(noise_sound)

    duration_clean_file = len(clean_file)
    
    extended_clean_file = clean_file
    while extended_clean_file.duration_seconds < noise_sound.duration_seconds:
        extended_clean_file += clean_file
    
    extended_clean_file = extended_clean_file[:len(noise_sound)]
    
    #extended_clean_file = match_target_amplitude(extended_clean_file, noise_sound.dBFS, offset_dB=5)
    
    combined = extended_clean_file.overlay(noise_sound)
    print(1)
    
    combined.export(save_folder + f"{count}.wav", format="wav")